# Logic

## Inputs
* Get positions with cost
* Get openorders
* Get current market price and last-day's low price

## Processing
### Compute price to close
* Get the DTE
* Compute _price-to-close_ as minumum of *quadratic equation* and last-day's low
* Quadratic equation is:
$103.6008 - 3.63457*x + 0.03454677*x*x$

### At the beginning of every trading day
* For every symbol in position, estimate the _price-to-close_
* Cancel all openorders
* Create open orders with _price-to-close_

### Logic to do the market is open
* Save openorder contracts
* Save filled contracts
* Save openpositions

In [1]:
# Harvesting program
# STATUS: Completed
# Runtime: 10 mins

#***          Start ib_insync (run once)       *****
#___________________________________________________

from ib_insync import *
util.startLoop()
ib=IB().connect('127.0.0.1', 7496, clientId=9) # kavi TWS live
# ib=IB().connect('127.0.0.1', 4001, clientId=9) # kavi IBG live

In [4]:
%%time
import pandas as pd
import numpy as np
import datetime

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
market = 'NSE'

#......     Get the positions   .......
p = ib.positions()

df_positions = pd.DataFrame([(i.contract.conId, i.position, i.avgCost) for i in p], 
             columns=['conId', 'position', 'avgCost'])


#.....     Get the contracts   .......
contracts = [Contract(conId=i.contract.conId, exchange='NSE') for i in p]
qual_contracts = ib.qualifyContracts(*contracts)

# make dataframe of qualified contracts
df_c = util.df([(c['Contract']) for c in util.tree(qual_contracts)])

# to put contracts as a new column in the dataframe, capture it
df_c1 = pd.DataFrame([c for c in qual_contracts])

df_contracts = pd.concat([df_c, df_c1], axis=1)

#.....   Get yesterday's ohlc   ........
hist_data = [ib.reqHistoricalData(contract=c, endDateTime='', durationStr='1 D', 
                      barSizeSetting='1 day', whatToShow='BID', useRTH=True) 
for i in range(0, len(qual_contracts), 50)
                    for c in qual_contracts[i:i+50]]

Wall time: 2.23 s


In [50]:
[x for x in hist_data if x != [] else 0]

SyntaxError: invalid syntax (<ipython-input-50-d3f48cf60662>, line 1)

In [56]:
for x in hist_data:
    if x != []:
        for h in x:
            print(h.date, h.open, h.high, h.low, h.close, h.average)
    else:
        print(np.nan)

2018-12-04 0.05 0.25 0.05 0.25 -1.0
2018-12-04 0.1 0.15 0.1 0.1 -1.0
2018-12-04 0.05 0.1 0.05 0.05 -1.0
2018-12-04 0.3 2.2 0.3 1.6 -1.0
2018-12-04 0.1 0.1 0.1 0.1 -1.0
2018-12-04 0.1 0.1 0.1 0.1 -1.0
2018-12-04 10.15 15.9 10.15 10.85 -1.0
nan
nan
2018-12-04 0.3 0.45 0.3 0.35 -1.0
nan
2018-12-04 0.15 3.5 0.15 3.5 -1.0
2018-12-04 0.05 0.45 0.05 0.15 -1.0
2018-12-04 0.05 1.2 0.05 1.2 -1.0
2018-12-04 0.05 0.4 0.05 0.2 -1.0
2018-12-04 0.05 0.1 0.05 0.1 -1.0
2018-12-04 0.05 1.5 0.05 1.25 -1.0
2018-12-04 0.15 0.15 0.1 0.1 -1.0
2018-12-04 0.2 0.9 0.2 0.25 -1.0
2018-12-04 0.05 0.2 0.05 0.15 -1.0
2018-12-04 0.15 0.2 0.15 0.2 -1.0
2018-12-04 4.25 6.9 4.25 6.2 -1.0
2018-12-04 2.75 5.8 2.75 5.5 -1.0
2018-12-04 0.5 0.9 0.5 0.65 -1.0
2018-12-04 0.2 0.3 0.2 0.2 -1.0
2018-12-04 2.65 8.65 2.65 6.0 -1.0
2018-12-04 0.2 0.6 0.2 0.5 -1.0
2018-12-04 0.15 3.2 0.15 3.2 -1.0
2018-12-04 0.15 0.3 0.15 0.15 -1.0
2018-12-04 1.45 5.45 1.45 3.4 -1.0
2018-12-04 0.2 0.5 0.2 0.35 -1.0
2018-12-04 0.45 3.5 0.45 1.65 -1.0


In [62]:
[((h.date, h.open, h.high, h.low, h.close, h.average) 
 if x != [] 
 for h in x 
 else np.nan)
 for x in hist_data]


SyntaxError: invalid syntax (<ipython-input-62-a102c1684bf7>, line 3)

In [44]:
df_hist = pd.DataFrame(pd.Series(hist_data),columns=['history'])

In [49]:
[print(h[0].date, h[0].open, h[0].high, h[0].low, h[0].close, h[0].average) for h in df_hist.history]

2018-12-04 0.05 0.25 0.05 0.25 -1.0
2018-12-04 0.1 0.15 0.1 0.1 -1.0
2018-12-04 0.05 0.1 0.05 0.05 -1.0
2018-12-04 0.3 2.2 0.3 1.6 -1.0
2018-12-04 0.1 0.1 0.1 0.1 -1.0
2018-12-04 0.1 0.1 0.1 0.1 -1.0
2018-12-04 10.15 15.9 10.15 10.85 -1.0


IndexError: list index out of range

In [ ]:
# make yesterday's dataframe
df_yesterday = pd.DataFrame([(h[0].date, h[0].open, h[0].high, h[0].low, h[0].close, h[0].average, h[0].volume) 
 for h in hist_data])
cols=['D', 'O', 'H', 'L', 'C', 'Avg', 'Vol']

In [ ]:
df_yesterday.columns = cols

#.....    Merge the dataframes and get target price  ....
df1 = pd.concat([df_positions, df_yesterday], axis=1)

In [18]:
df_yesterday

NameError: name 'df_yesterday' is not defined

In [15]:
df2 = df1.merge(df_contracts,how='left', on='conId')

IndexError: list index out of range

In [14]:
df2

NameError: name 'df2' is not defined

In [2]:
# get the DTE
df2['DTE'] = (pd.to_datetime(df2.lastTradeDateOrContractMonth) - datetime.datetime.now()).dt.days

# get expected price from DTE
def get_expPct(x):
    '''Gets the expected price percentage from DTE
    Assumes max DTE to be 30 days
    Arg: (x) as float
    Returns: expPrice as float
    Ref: http://interactiveds.com.au/software/Linest-poly.xls
    '''
    if x > 30:
        x = 30  # Forces the max DTE to be 30
    
    expPrice = 103.6008 - 3.63457*x + 0.03454677*x*x
    return expPrice/100

df2['expPrice'] = df2.DTE.apply(get_expPct)*df2.avgCost

# Set target price to be lower than yesterday's low and expPrice
# ...rounded to 0.05
df2['tgtPrice'] = df2[['L', 'expPrice']].min(axis=1).apply(lambda x: round(x*2,1)/2)

df2['order'] = [LimitOrder(action='BUY',totalQuantity=-position, lmtPrice=tgt_price)
            for position, tgt_price in zip(df2.position, df2.tgtPrice)]

ERROR:ib_insync.wrapper:Error 162, reqId 1435: Historical Market Data Service error message:HMDS query returned no data: MOTHERSUMI18DEC126.65PE@NSE Bid, contract: Contract(secType='OPT', conId=335436941, symbol='MOTHERSUM', lastTradeDateOrContractMonth='20181227', strike=126.65, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='MOTHERSUMI18DEC126.65PE', tradingClass='MOTHERSUMI')
ERROR:ib_insync.wrapper:Error 162, reqId 1436: Historical Market Data Service error message:HMDS query returned no data: RAMCOCEM18DEC530PE@NSE Bid, contract: Contract(secType='OPT', conId=335437965, symbol='RAMCOCEM', lastTradeDateOrContractMonth='20181227', strike=530.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='RAMCOCEM18DEC530PE', tradingClass='RAMCOCEM')
ERROR:ib_insync.wrapper:Error 162, reqId 1438: Historical Market Data Service error message:HMDS query returned no data: MOTHERSUMI18DEC123.35PE@NSE Bid, contract: Contract(secType='OPT', conId=33543693

IndexError: list index out of range

In [ ]:
#------------------------------------------------
# WARNING: THIS SECTION PLACES CLOSING TRADES
#________________________________________________

In [ ]:
# ....    Cancel all openorders   .....
cancels = ib.reqGlobalCancel()

In [ ]:
# ....   Place closing trades    .....
closingTrade = [ib.placeOrder(contract, order) for contract, order in zip(df2.contract, df2.order)]

In [ ]:
# Write closed orders placed to file
x  =  datapath + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_closing_orders.xlsx"
writer = pd.ExcelWriter(x)
df2.to_excel(writer, 'all-options', index=False, header=1)
writer.save()

In [3]:
df2

NameError: name 'df2' is not defined